In [1]:
import os
import hashlib

In [2]:
class NTT(object):   
    def __init__(self, n=16, q=None, base_inverse=False):
        if not n in [16,32,64,128,256,512,1024,2048]:
            raise ValueError("improper argument ",n)
        self.n = n  
        if not q:
            self.q = 1 + 2 * n
            while True:
                if (self.q).is_prime():
                    break
                self.q += 2 * n
        else:
            if q % (2 * n) != 1:
                raise ValueError("Valor de 'q' não verifica a condição NTT")
            self.q = q
             
        self.F = GF(self.q) ; self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen()
        
        g = (w ^ n + 1)
        x = g.roots(multiplicities=False)[-1]
        self.x = x
        if  base_inverse:
            rs = [x ^ (2 * i + 1)  for i in range(n)] 
            self.base = crt_basis([(w - r) for r in rs]) 
        else:
            self.base = None
    
    def ntt(self, f, inv=False):
        def _expand_(f): 
            u = f.list()
            return u + [0] * (self.n - len(u))      
            
        def _ntt_(x, N, f, inv=inv):
            if N == 1:
                return f
            N_  = N // 2 ; z = x ^ 2  
            f0  = [f[2 * i] for i in range(N_)] ; f1 = [f[2 * i + 1] for i in range(N_)] 
            ff0 = _ntt_(z, N_, f0, inv=inv) ; ff1 = _ntt_(z, N_, f1, inv=inv)  
    
            s  = self.F(1) if inv else x 
            ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a     = ff0[i] ; b          = s * ff1[i]  
                ff[i] = a + b  ; ff[i + N_] = a - b 
                s     = s * z                    
            return ff 

        vec = _expand_(f)
        if  not inv:
            return self.R(_ntt_(self.x, self.n, vec, inv=inv))
        elif self.base != None:
            return sum([vec[i] * self.base[i] for i in range(self.n)])
        else:
            n_ = (self.F(self.n)) ^ -1
            x_ = (self.x) ^ -1 
            u  = _ntt_(x_, self.n, vec, inv=inv)
            return self.R([n_ * x_ ^ i * u[i] for i in range(self.n)])
 
    def random_pol(self,args=None):
        return (self.R).random_element(args)

In [3]:
class KYBER():
    def __init__(self):
        # Specific Parameters 
        self.n  = 256
        self.nl = 9
        self.q  = 3329

        # Extra Parameters
        self.k  = 2
        self.n1 = 3
        self.n2 = 2
        self.du = 10
        self.dv = 4

        # Ring, Vector and Matrix
        R = PolynomialRing(ZZ, 'a') ; a = R.gen()
        self.RingRq   = R.quotient(a ^ self.n + 1, 'x') ; self.x = self.RingRq.gen()
        self.VectorRq = MatrixSpace(self.RingRq, self.k, 1)
        self.MatrixRq = MatrixSpace(self.RingRq, self.k, self.k)
    

    # ByteToBits Function
    # Takes as input an array of l bytes and produces as output an array of 8l bits.
    def bytes_to_bits(self, byte_array):
        bit_array = []
        byte_array_length = len(byte_array) * 8
        for i in range(byte_array_length):
            base  = int(i // 8)
            shift = int(i % 8)
            bit_array.append( byte_array[base] >> shift & 0x1 )
        return bit_array

    def byteArrToBytes(self, btArray) :
        byts = b''
        for i in btArray :
            byts += int(i).to_bytes(1,'little')
        return byts
        

    # Compress Function
    # Takes an element from RingRq and outputs an element in {0, ..., 2 ^ d - 1}. Where d < [log2(q)]
    def compress_small(self, x, d):
        return round( ((2 ^ d) / self.q) * x ) % (2 ^ d) 

    # It gives back a list of coeficients
    # For polinomials
    def compress(self, x, d):
        res   = [] 
        coefs = x.list()
        for coef in coefs:
            res.append( self.compress_small(coef, d) )     
        return self.RingRq(res)

    # It gives back a list of polinomials
    # For vectors
    def compress_big(self, x, d):
        res   = [] 
        coefs = x.list()
        for coef in coefs:
            res.append( self.RingRq(self.compress(coef, d)) )     
        return self.VectorRq(res)

    # Decompress Function
    # x' = Decompress(Compress(x, d), d) and x' is close to x.
    def decompress_small(self, x, d):
        return round( (self.q / (2 ^ d))  * x )

    # It gives back a list of coeficients
    # For polinomials
    def decompress(self, x, d):
        res   = [] 
        coefs = x.list()
        for coef in coefs:
            res.append( self.decompress_small(coef, d) )     
        return self.RingRq(res)

    # It gives back a list of polinomials
    # For vectors
    def decompress_big(self, x, d):
        res   = [] 
        coefs = x.list()
        for coef in coefs:
            res.append( self.RingRq(self.decompress(coef, d)) )     
        return self.VectorRq(res)


    # Parse Function
    # Receives a byte stream and computes the NTT representation. This function assumes q = 33229 
    def parse(self, byte_array):
        i = 0 ; j = 0 ; 
        a = [0] * self.n
        while j < self.n:
            d1 = byte_array[i] + 256 * (byte_array[i + 1] % 16)
            d2 = round(byte_array[i + 1] / 16) + 16 * byte_array[i + 2]
            if d1 < self.q:
                a[j] = d1
                j += 1
            if  d2 < self.q and j < self.n:
                a[j] = d2
                j += 1
            i += 3
        return self.RingRq(a)


    # CBD Function
    # how a polynomial f from Rq is sampled according to Bn deterministically from 64n bytes of output of 
    # a pseudorandom function (we fix n = 256 in this description).
    def cbd(self, byte_array, n):
        bit_array = self.bytes_to_bits(byte_array)
        f = []
        for i in range(256):
            a = 0 ; b = 0
            for j in range(n):
                a = a + bit_array[2 * i * n + j]
                b = b + bit_array[2 * i * n + n + j]
            f.append(a - b) 
            return self.RingRq(f)

    
    # Decode Function
    def decode(self, byte_array, l): # Despite giving l as an argument, we assume l = 12 
        f = []
        for i in range(self.n / 2):
            f.append( ((byte_array[3 * i + 0] >> 0) | (byte_array[3 * i + 1] << 8)) & 0xff )
            f.append( ((byte_array[3 * i + 1] >> 4) | (byte_array[3 * i + 2] << 4)) & 0xff )
        return self.RingRq(f)
    
    def decode_vector(self, byte_array, l):
        size = len(byte_array) // (32 * l)
        f    = [None] * self.k
        for i in range(size):
            f[i] = self.decode( byte_array[i * 32 * l : (i + 1) * 32 * l], l )
        return self.VectorRq(f)
        
    def encode(self, f, l):
        byte_array         = []
        f_coeficients_csuq = []
        f_coeficients      = f.list()
        t0 = 0 ; t1 = 0

        for i in range(self.n):
            a = f_coeficients[i]
            a -= self.q
            a += (a >> 15) & self.q
            f_coeficients_csuq.append(a)

        for i in range(self.n / 2):
            t0 = f_coeficients_csuq[2 * i]
            t1 = f_coeficients_csuq[2 * i + 1]

            byte_array.append( (t0 >> 0) )
            byte_array.append( (t0 >> 8) | (t1 << 4) )
            byte_array.append( (t1 >> 4) )

        return byte_array

    # Encode Function For Vectors
    def encode_vector(self, vector, l):
        poli_list  = vector.list()
        byte_array = []
        for i in range(self.k):
            byte_array += self.encode(poli_list[i], l)
        return byte_array


    # This next two functions do the same as encode and decode but for 32 bytes messages
    def decode_32(self, message):
        f    = []
        mask = 0
        for i in range(self.n / 8):
            for j in  range(8):
                mask = -((message[i] >> j) & 1)
                f.append( mask & ((self.q + 1) // 2) )
        return self.RingRq(f) 

    def encode_32(self, poli):
        byte_array         = [None] * 32 
        f_coeficients_csuq = []
        f_coeficients      = poli.list()
        t = 0

        for i in range(self.n):
            a = f_coeficients[i]
            a -= self.q
            a += (a >> 15) & self.q
            f_coeficients_csuq.append(a)

        for i in range(self.n / 8):
            byte_array[i] = 0
            for j in  range(8):
                t = (((f_coeficients_csuq[8 * i + j] << 1) + self.q / 2) / self.q) & 1    
                byte_array[i] |= t << j 
        return byte_array 
    
    def decode_vector_32(self, byte_array, l):
        size = len(byte_array) // (32 * l)
        f    = [None] * self.k
        for i in range(size):
            f[i] = self.decode_32( byte_array[i * 32 * l : (i + 1) * 32 * l] )
        return self.VectorRq(f)


    # XOF function
    # extendable output function with SHAKE-128
    def xof(self, data, i, j):
        hash_funtion = hashlib.shake_128()
        hash_funtion.update(data + j.to_bytes(4, "little") + i.to_bytes(4, "little"))
        return hash_funtion.digest(self.q) # Is it self.q ? 

    # PRF Function
    # Pseuddorandom function with SHAKE-256
    def prf(self, data, N, n):
        Nb     = int(N).to_bytes(4, "little")
        seed   = data + Nb

        hash_funtion = hashlib.shake_256()
        hash_funtion.update(seed)
        return hash_funtion.digest(self.q)

    # KDF function
    # Key derivation Function with SHAKE-256
    def kdf(self, data, length):
        hash_funtion = hashlib.shake_256()
        hash_funtion.update(data)
        return hash_funtion.digest(length)
    
    # Hash Function H
    # Hash Function withwith SHA3-256
    def hash_H(self, data):
        hash_funtion = hashlib.sha3_256()
        hash_funtion.update(data)
        return hash_funtion.digest()

    # Hash Function G
    # Hash Function withwith SHA3-512
    def hash_G(self, data):
        hash_funtion = hashlib.sha3_512()
        hash_funtion.update(data)
        return hash_funtion.digest()

In [4]:
class KYBER_PKE(KYBER):
    def key_generation_pke(self):
        d = os.urandom(32)
        N = 0
        hashed    = self.hash_G(d)
        ro, sigma = hashed[:32], hashed[32:]

        # Generate Matrix A
        A0 = [None] * self.k * self.k
        for i in range(self.k):
            for j in range(self.k):
                byte_stream        = self.xof(ro, i, j)
                A0[i * self.k + j] = self.parse(byte_stream)
        A = self.MatrixRq(A0)
        
        # Generate Vec s
        s = [None] * self.k
        for i in range(self.k):
            byte_array = self.prf(sigma, N, self.n1)
            s[i]       = self.cbd(byte_array, self.n1)
            N = N + 1

        # Generate Vec e
        e = [None] * self.k
        for i in range(self.k):
            byte_array = self.prf(sigma, N, self.n1)
            e[i]       = self.cbd(byte_array, self.n1)
            N = N + 1
        
        nTT   = NTT()
        s_ntt = self.VectorRq( [nTT.ntt(s[i]) for i in range(self.k)] )
        e_ntt = self.VectorRq( [nTT.ntt(e[i]) for i in range(self.k)] )
        
        t = self.compress_big((A * s_ntt) + e_ntt, 11)
    
        public_key = self.encode_vector(t, 12) + list(ro)
        secret_key = self.encode_vector(s_ntt, 12)
        return public_key, secret_key

    def encryption_pke(self, public_key, message, coins):
        N = 0
        
        t  = self.decode_vector(public_key, 12)
        ro = bytes(public_key[12 * self.k * self.n / 8:])

        # Generate Matrix A
        A_empty = [None] * self.k * self.k # We instanciate A
        for i in range(self.k):
            for j in range(self.k):
                byte_stream             = self.xof(ro, i, j)
                A_empty[i * self.k + j] = self.parse(byte_stream)
        A  = self.MatrixRq(A_empty)
        At = A.transpose()

        # Generate Vec r
        r = [None] * self.k
        for i in range(self.k):
            byte_array = self.prf(coins, N, self.n1)
            r[i]       = self.cbd(byte_array, self.n1)
            N = N + 1

        # Generate Vec e1
        e1 = [None] * self.k
        for i in range(self.k):
            byte_array  = self.prf(coins, N, self.n2)
            e1[i]       = self.cbd(byte_array, self.n2)
            N = N + 1
        
        e2_byte_array = self.prf(coins, N, self.n2)
        e2 = self.cbd(e2_byte_array, self.n2)

        nTT = NTT()
        r_ntt  = self.VectorRq( [nTT.ntt(r[i]) for i in range(self.k)] )
        e1_ntt = self.VectorRq( e1 )

        ut = (At * r_ntt) 
        u  = self.VectorRq( [nTT.ntt(ut[i][0], inv=True) for i in range(self.k)] ) + e1_ntt
        vt = (t.transpose() * r_ntt)
        v  = self.RingRq( nTT.ntt(vt[0][0], inv=True) ) + e2 + self.decompress( self.decode_32(message) , 1)

        c1 = self.encode_vector( self.compress_big(u, self.du), self.du )
        c2 = self.encode( self.compress(v, self.dv), self.dv )
        return c1 + c2

    def decryption_pke(self, secret_key, cipher_text):
        c1 = cipher_text[:self.du * self.k * self.n / 8]
        c2 = cipher_text[self.dv * self.k * self.n / 8:]

        u = self.decompress_big( self.decode_vector_32(c1, self.du), self.du )
        v = self.decompress( self.decode(c2, self.dv), self.dv )
        
        s = self.decode_vector(secret_key, 12)

        nTT = NTT()
        u_ntt = self.VectorRq( [nTT.ntt(u.list()[i]) for i in range(self.k)] )

        message = self.encode(self.compress(v - self.RingRq(nTT.ntt((s.transpose() * u_ntt)[0][0], inv=True)), 1), 1)
        return message
        

In [5]:
kyber = KYBER()

In [6]:
f1 = kyber.RingRq([255] * 255)
f2 = kyber.RingRq([500] * 255)
v  = kyber.VectorRq((f1 , f2)) 

#kyber.decode_vector(kyber.encode_vector(v, 12),12)

In [7]:
r       = os.urandom(kyber.q)
coins   = os.urandom(32)
message = os.urandom(32)
print(message)

b"FM\x1eX\xcaT\xa9\xfa8\xf9\x07F\xd0BM=\xc0\x1d\xd7M'eTmF\xfc*\xad\xf0B\xe0\xaf"


In [8]:
pke = KYBER_PKE()
pk, sk = pke.key_generation_pke()
ct = pke.encryption_pke(pk, message, coins)
m  = pke.decryption_pke(sk, ct)


print("public_key length: ", len(pk))
print("secret_key length: ", len(sk))
print(m)



public_key length:  800
secret_key length:  768
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
class KYBER_KEM(KYBER_PKE):
    def __int_to_bytes(self, x):
        bt = b""
        for i in range( len(x) ):
            bt += int(x[i]).to_bytes((int(x[i]).bit_length() + 7) // 8, 'big')
        return bt
    
    def key_generation_kem(self):
        z = os.urandom(32)

        pk, sk_pke = self.key_generation_pke()
        sk = self.__int_to_bytes(sk_pke) + self.__int_to_bytes(pk) + self.hash_H( self.__int_to_bytes(pk) ) + z

        return pk, sk

    def encapsulation(self, public_key, message):
        m = self.hash_H(message)
        kr  = self.hash_G(m + self.hash_H( self.__int_to_bytes(public_key) )) 
        k = kr[:32]
        r = kr[32:]

        c = self.encryption_pke(public_key, m, r)
        K = self.kdf(k + self.hash_H(self.__int_to_bytes(c)), 32)
        return c, K

    def decapsulation(self, ciphertext, sk):
        pk = sk[12 * self.k * self.n / 8:]
        h  = sk[24 * self.k * self.n / 8 + 32:] 
        z  = sk[24 * self.k * self.n / 8 + 64:] 
        
        m = self.decryption_pke(self.byteArrToBytes(sk), ciphertext)
        
        kr = self.hash_sha512(m + h)
        k, r = kr[:32], kr[32:]
        
        c_ = self.encryption_pke(pk, m , r)
        
        if c == c_:
            K = self.kdf(k + self.self.hash_sha256(c))
        else:
            K = self.kdf(z + self.self.hash_sha256(c))
        return K
        

In [10]:
kem = KYBER_KEM()
pk, sk = kem.key_generation_kem()
c, K = kem.encapsulation(pk, message)
K = kem.decapsulation(c, sk)

IndexError: list assignment index out of range